In [6]:
from ezpymysql import Connection

db = Connection(
    'localhost',
    'crawler',
    'root',
    'root'
)

# 若用pymysql库，需要cursor.execute(sql) 来执行sql语句
# cursor = db.cursor()



# 此表用于存储hub页面的url,hub页面就是含有大量新闻链接、不断更新的网页

# AUTO_INCREMENT定义列为自增的属性，一般用于主键，数值会自动加1。
# ENGINE 设置存储引擎，CHARSET 设置编码

sql_create = """CREATE TABLE `crawler_hub` (
              `id` int(10) unsigned NOT NULL AUTO_INCREMENT,
              `url` varchar(64) NOT NULL,
              `created_at` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
              PRIMARY KEY (`id`),
              UNIQUE KEY `url` (`url`)
            ) ENGINE=MyISAM DEFAULT CHARSET=utf8"""

t1 = db.execute(sql_create)
print(t1)

Traceback (most recent call last):
  File "C:\Users\jiege\PycharmProjects\crawl（猿人学python）\大规模异步新闻爬虫5：实现一个同步定向新闻爬虫\ezpymysql\__init__.py", line 114, in execute
    cursor.execute(query, kwparameters or parameters)
  File "D:\Anacondaaa\envs\qqqq\lib\site-packages\pymysql\cursors.py", line 148, in execute
    result = self._query(query)
  File "D:\Anacondaaa\envs\qqqq\lib\site-packages\pymysql\cursors.py", line 310, in _query
    conn.query(q)
  File "D:\Anacondaaa\envs\qqqq\lib\site-packages\pymysql\connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "D:\Anacondaaa\envs\qqqq\lib\site-packages\pymysql\connections.py", line 775, in _read_query_result
    result.read()
  File "D:\Anacondaaa\envs\qqqq\lib\site-packages\pymysql\connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "D:\Anacondaaa\envs\qqqq\lib\site-packages\pymysql\connections.py", line 725, in _read_packet
    packet.raise

OperationalError: (1050, "Table 'crawler_hub' already exists")

In [7]:

# 此表存储html内容
# html是大量的文本内容，压缩存储会大大减少磁盘使用量。这里，我们选用lzma压缩算法

sql_create2 = '''CREATE TABLE `crawler_html` (
              `id` bigint(20) unsigned NOT NULL AUTO_INCREMENT,
              `urlhash` bigint(20) unsigned NOT NULL COMMENT 'farmhash',
              `url` varchar(512) NOT NULL,
              `html_lzma` longblob NOT NULL,
              `created_at` timestamp NULL DEFAULT CURRENT_TIMESTAMP,
              PRIMARY KEY (`id`),
              UNIQUE KEY `urlhash` (`urlhash`)
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8'''
t2 = db.execute(sql_create2)
print(t2)

0


In [9]:
import urllib.parse as urlparse
import lzma
import pyfarmhash
import traceback


from ezpymysql import Connection
from urlpool import UrlPool
import functions as fn
import config

class NewsCrawlerSync:
    def __init__(self, name):
        self.db = Connection(
            config.db_host,
            config.db_db,
            config.db_user,
            config.db_password
        )
        self.logger = fn.init_file_logger(name + '.log')
        self.urlpool = UrlPool(name)
        self.hub_hosts = None
        self.load_hubs()

    def load_hubs(self,):
        sql = 'select url from crawler_hub'
        data = self.db.query(sql)
        self.hub_hosts = set()
        hubs = []
        for d in data:
            host = urlparse.urlparse(d['url']).netloc
            self.hub_hosts.add(host)
            hubs.append(d['url'])
        self.urlpool.set_hubs(hubs, 300)

    def save_to_db(self, url, html):
        urlhash = farmhash.hash64(url)
        sql = 'select url from crawler_html where urlhash=%s'
        d = self.db.get(sql, urlhash)
        if d:
            if d['url'] != url:
                msg = 'farmhash collision: %s <=> %s' % (url, d['url'])
                self.logger.error(msg)
            return True
        if isinstance(html, str):
            html = html.encode('utf8')
        html_lzma = lzma.compress(html)
        sql = ('insert into crawler_html(urlhash, url, html_lzma) '
               'values(%s, %s, %s)')
        good = False
        try:
            self.db.execute(sql, urlhash, url, html_lzma)
            good = True
        except Exception as e:
            if e.args[0] == 1062:
                # Duplicate entry
                good = True
                pass
            else:
                traceback.print_exc()
                raise e
        return good

    def filter_good(self, urls):
        goodlinks = []
        for url in urls:
            host = urlparse.urlparse(url).netloc
            if host in self.hub_hosts:
                goodlinks.append(url)
        return goodlinks

    def process(self, url, ishub):
        status, html, redirected_url = fn.downloader(url)
        self.urlpool.set_status(url, status)
        if redirected_url != url:
            self.urlpool.set_status(redirected_url, status)
        # 提取hub网页中的链接, 新闻网页中也有“相关新闻”的链接，按需提取
        if status != 200:
            return
        if ishub:
            newlinks = fn.extract_links_re(redirected_url, html)
            goodlinks = self.filter_good(newlinks)
            print("%s/%s, goodlinks/newlinks" % (len(goodlinks), len(newlinks)))
            self.urlpool.addmany(goodlinks)
        else:
            self.save_to_db(redirected_url, html)

    def run(self,):
        while 1:
            urls = self.urlpool.pop(5)
            for url, ishub in urls.items():
                self.process(url, ishub)


if __name__ == '__main__':
    crawler = NewsCrawlerSync('yuanrenxyue')
    crawler.run()

ModuleNotFoundError: No module named 'farmhash'